In [6]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import os
import numpy as np

matplotlib.rcParams.update({'figure.figsize':(14,10), 'font.size':14})

In [7]:
dataFolder = '../data'
files = [x for x in os.listdir(dataFolder) if x.endswith('.csv')]
print files

['0.csv', '0Rep.csv', '1.csv', '1Rep.csv', '2.csv', '2Rep.csv', '3.csv', '3Rep.csv', '4.csv', '4Rep.csv', '5.csv', '5Rep.csv', '6.csv', '6Rep.csv', '7.csv', '7Rep.csv', '8.csv', '8Rep.csv', '9.csv', '9Rep.csv', 'chdown.csv', 'chdownRep.csv', 'chup.csv', 'chupRep.csv', 'enter.csv', 'enterRep.csv', 'info.csv', 'infoRep.csv', 'lang.csv', 'langRep.csv', 'last.csv', 'lastRep.csv', 'mute.csv', 'muteRep.csv', 'power.csv', 'powerRep.csv', 'voldown.csv', 'voldownRep.csv', 'volup.csv', 'volupRep.csv']


In [8]:
traces = {}
for f in files:
    fName = os.path.join(dataFolder, f)
    with open(fName) as fileIn:
        key = f.split('.')[0]
        d = np.loadtxt(fName, skiprows=1, delimiter=',')
        leadingSelect = d[:,1] == 0
        leading = d[leadingSelect, 0]
        traces[key] = leading

In [10]:
deltas = []

def processPacket(diffs, verbose=False):
    if verbose:
        print 'Diffs:'
        print diffs
    
    # Sanity Checking
    if len(diffs) != 17:
        if verbose:
            print '!!!!!!!!! Wrong Length'
        return None

    timingInfo = diffs[:8]
    buttonInfo = diffs[9:]
    
    minSpace = diffs[2]
    maxSpace = diffs[3]
    halfSpaces = diffs[4:6]
    oneBelowHalfSpace = diffs[6]
    oneBelowFullSpace = diffs[7]
    
    if (maxSpace - np.sum(halfSpaces) > .1 or # if the sum of half spaces isn't within 100 microseconds
        oneBelowFullSpace - halfSpaces[-1] - oneBelowHalfSpace > .1):
        if verbose:
            print '!!!!!!!!! Invalid Timing Packet'
        return None    
    
    spacing = (maxSpace - minSpace) / 15
    
    buttonLevels = buttonInfo / spacing
    if verbose:
        print 'Button Levels:'
        print buttonLevels
    if np.max(np.abs((buttonLevels - np.round(buttonLevels)))) > .4: # If any of the levels are close to being 
                                                                     # undecideable, skip it
        if verbose:
            print '!!!!!!!!! Undecideable Button Info'
        return None

    buttonDetectedLevels = np.round(buttonLevels) - 7
    if verbose:
        print 'Quantized Button Levels:'
        print buttonDetectedLevels
    if np.max(buttonDetectedLevels) > 15 or np.min(buttonDetectedLevels) < 0:
        if verbose:
            print '!!!!!!!!! Invalid Button Level'
        return None
    return buttonDetectedLevels

def decodeKey(levels):
    # Sanity Checks
    if np.sum(levels) % 16 != 0:
        return None, False
    if len(levels) != 8:
        return None, False

    isRepeat = False
    if levels[2] == 8:
        isRepeat = True
    
    ret = None
    val = levels[5]
    if levels[4] == 0:
        if val < 10:
            ret = str(val)
        elif val == 10:
            ret = 'volup'
        elif val == 11:
            ret = 'voldown'
        elif val == 12:
            ret = 'mute'
        elif val == 13:
            ret = 'chup'
        elif val == 14:
            ret = 'chdown'
        elif val == 15:
            ret = 'power'
    else:
        if val == 1:
            ret = 'last'
        if val == 2:
            ret = 'lang'
        if val == 5:
            ret = 'enter'
        if val == 6:
            ret = 'info'

    return ret, isRepeat
    
    pass
#     print levels

residuals = np.array([])
buttonDict = {}
for key in [x for x in sorted(traces.keys()) if 'Rep' not in x]:
#     print 'Key:', key
#     print len(traces[key])
    for i in range(0, len(traces[key]) / 18):
        leading = traces[key][i * 18:(i + 1) * 18]
        diffs = np.diff(leading) * 1000 # convert to ms
        print diffs
        levels = processPacket(diffs)
        buttonDict[key] = levels
        print decodeKey(levels)

#         spacing = (diffs[4] + diffs[5]) / 2.0 - diffs[6]
#         print 'Key:',key
#         print diffs[:8]

#         minSpace = diffs[2]
#         maxSpace = diffs[3]
#         deltaSpace = diffs[5] - diffs[6]
#         print deltaSpace

#         print maxSpace - minSpace

#         print (maxSpace - minSpace) / deltaSpace

#         data = diffs[9:] / (maxSpace - minSpace) * 15
#     #     data = diffs[9:] / deltaSpace
#         print data
#         residuals = np.hstack([data - np.round(data), residuals])
    
# _ = plt.hist(residuals, 20)
for k in sorted(buttonDict.keys()):
    print ' '.join(['{0:2d} '.format(int(x)) for x in buttonDict[k]]), k

[  1.10119   1.93754   0.96451   3.04304   1.51928   1.51939   1.37848
   2.93206  13.17451   1.10541   3.04307   0.96453   0.96467   0.96464
   0.96453   0.9646    0.96462]
('0.0', False)
[  1.10108   1.96308   0.93882   3.04287   1.51909   1.51919   1.37846
   2.90165  13.20784   1.10101   2.90625   0.96449   0.9645    0.96452
   1.10529   0.96449   0.96444]
('1.0', False)
[  1.10547   1.95908   0.93903   3.03887   1.52364   1.51931   1.37853
   2.90216  13.20528   1.10547   2.7658    0.96466   0.96457   0.96449
   1.24207   0.96468   0.96465]
('2.0', False)
[  1.10099   1.93742   0.96439   3.04268   1.51921   1.51923   1.37845
   2.90648  13.20417   1.10112   2.62887   0.96444   0.96445   0.96452
   1.38279   0.96451   0.96458]
('3.0', False)
[  1.10068   1.937     0.96417   3.04158   1.51863   1.51868   1.38214
   2.90088  13.20788   1.10054   2.48687   0.96837   0.96405   0.96418
   1.51858   0.96827   0.95977]
('4.0', False)
[  1.1012    1.93758   0.9645    3.03853   1.5193    1.

In [176]:

for key in sorted([x for x in traces.keys() if 'Rep' in x]):
    print 'Key:', key
    print 'Num Packets:', len(traces[key]) / 18
    for i in range(0, len(traces[key]) / 18):

        leading = traces[key][i*18:(i+1)*18]
        diffs = np.diff(leading) * 1000 # convert to ms
        levels = processPacket(diffs)
        print levels
        print decodeKey(levels)


Key: 0Rep
Num Packets: 5
[  1.  15.   0.   0.   0.   0.   0.   0.]
('0.0', False)
[ 1.  7.  8.  0.  0.  0.  0.  0.]
('0.0', True)
[ 1.  7.  8.  0.  0.  0.  0.  0.]
('0.0', True)
[ 1.  7.  8.  0.  0.  0.  0.  0.]
('0.0', True)
[ 1.  7.  8.  0.  0.  0.  0.  0.]
('0.0', True)
Key: 1Rep
Num Packets: 5
[  1.  14.   0.   0.   0.   1.   0.   0.]
('1.0', False)
[ 1.  6.  8.  0.  0.  1.  0.  0.]
('1.0', True)
[ 1.  6.  8.  0.  0.  1.  0.  0.]
('1.0', True)
[ 1.  6.  8.  0.  0.  1.  0.  0.]
('1.0', True)
[ 1.  6.  8.  0.  0.  1.  0.  0.]
('1.0', True)
Key: 2Rep
Num Packets: 5
[  1.  13.   0.   0.   0.   2.   0.   0.]
('2.0', False)
[ 1.  5.  8.  0.  0.  2.  0.  0.]
('2.0', True)
[ 1.  5.  8.  0.  0.  2.  0.  0.]
('2.0', True)
[ 1.  5.  8.  0.  0.  2.  0.  0.]
('2.0', True)
[ 1.  5.  8.  0.  0.  2.  0.  0.]
('2.0', True)
Key: 3Rep
Num Packets: 5
[  1.  12.   0.   0.   0.   3.   0.   0.]
('3.0', False)
[ 1.  4.  8.  0.  0.  3.  0.  0.]
('3.0', True)
[ 1.  4.  8.  0.  0.  3.  0.  0.]
('3.0', True)
